#### Loading Data

In [5]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
suppressPackageStartupMessages(source("brain_simulation_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/tree_analysis/tree_filter.R"))
suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(ggpubr))
suppressPackageStartupMessages(library(dplyr))
load("environment/Chimp/se_chimp.RData")
load("environment/Chimp/mode=gcbias/post_type=gibbs_nrep=100_tf=100/seChimpScaled.RData")
load("environment/Chimp/mapDf.RData")
load("environment/Chimp/mode=gcbias/post_type=gibbs_nrep=100_tf=100/tree_analysis/trMeanCorUPGMA.RData")

In [3]:
sampDorNucl <- data.frame(colData(seChimp)) %>% filter(tissue == "medial dorsal nucleus of thalamus") %>% dplyr::select(names) %>% unlist
seChimp <- seChimp[,!(colnames(seChimp) %in% sampDorNucl)]

In [10]:
rest <- colData(seChimp)[["names"]][colData(seChimp)[["condition"]] == "Rest"]
cere <- colData(seChimp)[["names"]][colData(seChimp)[["condition"]] == "Cere"]

In [12]:
median(colSums(assays(seChimp)[["counts"]][,rest]))
median(colSums(assays(seChimp)[["counts"]][,cere]))
median(colSums(assays(seChimp)[["infRep1"]][,rest]))
median(colSums(assays(seChimp)[["infRep1"]][,cere]))
median(colSums(assays(seChimp)[["infRep2"]][,rest]))
median(colSums(assays(seChimp)[["infRep2"]][,cere]))

median(colSums(assays(seChimpScaled)[["infRep1"]][,rest]))
median(colSums(assays(seChimpScaled)[["infRep1"]][,cere]))
median(colSums(assays(seChimpScaled)[["infRep2"]][,rest]))
median(colSums(assays(seChimpScaled)[["infRep2"]][,cere]))

[1] 15121526

[1] 15002460

[1] 15121526

[1] 15002460

[1] 15121526

[1] 15002460

[1] 13733313

[1] 12374491

[1] 13735401

[1] 12385792

#### Reading all the nwk files for the trees

In [13]:
dir <- "/fs/cbcb-lab/rob/students/noor/Uncertainity/ChimpData/term_out/mode_gcbias=True/posttype=gibbs_npost=100_tf=100/tree_analysis"
treesCons0 <- read.tree(file.path(dir, "nothr0", "cluster_nwk.txt"))
txpsCons0 <- rownames(seChimp)[as.numeric(unlist(lapply(treesCons0, function(tr) tr$tip)))+1]

treesCons <- read.tree(file.path(dir, "nothr", "cluster_nwk.txt"))
txpsCons <- rownames(seChimp)[as.numeric(unlist(lapply(treesCons, function(tr) tr$tip)))+1]

treesMeanInf0 <- read.tree(file.path(dir, "mean_inf0", "group_nwk.txt"))
txpsMeanInf0 <- rownames(seChimp)[as.numeric(unlist(lapply(treesMeanInf0, function(tr) tr$tip)))+1]

treesThr <- read.tree(file.path(dir, "thr", "cluster_nwk.txt"))
txpsThr <- rownames(seChimp)[as.numeric(unlist(lapply(treesThr, function(tr) tr$tip)))+1]
                                                       
groupsThr <- read.delim(file.path(dir, "old", "PTB162LA1C", "clusters.txt"), header = F)
txpsOld <- unlist(sapply(strsplit(groupsThr$V1, split=",", fixed=T), function(x) x[2:length(x)]))
txpsOld <- gsub("\\.[0-9]+", "", txpsOld)
txpsCor <- trMeanCorUPGMA$tip

In [14]:
se <- labelKeep(seChimp)

In [15]:
length(txpsThr)
length(txpsOld)
length(txpsMeanInf0)
length(txpsCons0)
length(txpsCons)
length(txpsCor)
length(rownames(se)[mcols(se)[["keep"]]])

[1] 15773

[1] 5428

[1] 39856

[1] 39491

[1] 37858

[1] 35114

[1] 31185

In [16]:
txpsAll <- Reduce(union,list(txpsCons0, txpsCons, txpsThr, txpsMeanInf0, txpsOld, txpsCor, rownames(se)[mcols(se)[["keep"]]]))
length(txpsAll)

[1] 45375

#### Consensus Tree

In [31]:
treeCons <- mergeTree(treesCons, se = mapDf)
mb <- mergeLeaves(treeCons, seChimp[txpsAll,])
treeCons <- mb[["tree"]]
yConsNS <- mb[["ySwish"]]
rm(mb)
gc()
sum(rownames(yConsNS) != treeCons$tip)
yAggConsNS <- prepSwish(treeCons, seChimp[treeCons$tip,])
yAggConsNS <- fishpond::computeInfRV(yAggConsNS, meanVariance=F)
yAggCons <- prepSwish(treeCons, seChimpScaled[treeCons$tip,])
infReps <- assays(yAggCons)[grep("infRep", assayNames(yAggCons))]
infReps <- abind::abind(as.list(infReps), along = 3)
mcols(yAggConsNS)[["LFC"]] <- getLog2FC(infReps, colData(yAggCons)[["condition"]])
rm(yAggCons)
gc()
save(yAggConsNS, file="environment/Chimp/mode=gcbias/post_type=gibbs_nrep=100_tf=100/tree_analysis/yAggConsNS.RData")
save(treeCons, file="environment/Chimp/mode=gcbias/post_type=gibbs_nrep=100_tf=100/tree_analysis/treeCons.RData")

[1] "Missing txps 7517"


used       (Mb)    gc trigger (Mb)    max used   (Mb)   
Ncells   10670779   569.9   27719894  1480.5   27719894  1480.5
Vcells 5058376664 38592.4 8794373681 67095.8 8048195828 61402.9

[1] 0

used       (Mb)    gc trigger  (Mb)    max used   (Mb)   
Ncells   10671622   570.0    27719894  1480.5   27719894  1480.5
Vcells 5606784538 42776.4 10553328417 80515.6 9887100224 75432.6

In [ ]:
# desc <- Descendants(treeCor, length(treeCor$tip)+1:treeCor$Nnode)
# fourInds <- which(sapply(desc, length)==4)
# length(fourInds)
# fourIndsNUniq <- fourInds[sapply(fourInds, function(inds) length(unique(mapDf[treeCor$tip[desc[[inds]]],"gene_id"])) != 1)]
# length(fourIndsNUniq)

In [ ]:
# i <- 28
# mapDf[treeCor$tip[desc[[fourIndsNUniq[i]]]],"gene_id"]
# txps <- treeCor$tip[desc[[fourIndsNUniq[i]]]]
# corMean[txps[1], txps]
# min(corMean[txps[1], ])

#### Consensus Tree 0

In [30]:
treeCons0 <- mergeTree(treesCons0, se = mapDf)
mb <- mergeLeaves(treeCons0, seChimp[txpsAll,])
treeCons0 <- mb[["tree"]]
yConsNS0 <- mb[["ySwish"]]
rm(mb)
gc()
sum(rownames(yConsNS0) != treeCons0$tip)
yAggConsNS0 <- prepSwish(treeCons0, seChimp[treeCons0$tip,])
yAggConsNS0 <- fishpond::computeInfRV(yAggConsNS0, meanVariance=F)
yAggCons0 <- prepSwish(treeCons0, seChimpScaled[treeCons0$tip,])

infReps <- assays(yAggCons0)[grep("infRep", assayNames(yAggCons0))]
infReps <- abind::abind(as.list(infReps), along = 3)
mcols(yAggConsNS0)[["LFC"]] <- getLog2FC(infReps, colData(yAggCons0)[["condition"]])
rm(yAggCons0)
gc()

save(yAggConsNS0, file="environment/Chimp/mode=gcbias/post_type=gibbs_nrep=100_tf=100/tree_analysis/yAggConsNS0.RData")
save(treeCons0, file="environment/Chimp/mode=gcbias/post_type=gibbs_nrep=100_tf=100/tree_analysis/treeCons0.RData")

[1] "Missing txps 5884"


used       (Mb)    gc trigger (Mb)    max used   (Mb)   
Ncells   10669151   569.8   27719894  1480.5   27719894  1480.5
Vcells 4144684381 31621.5 7328578068 55912.7 7325005957 55885.4

[1] 0

used       (Mb)    gc trigger (Mb)    max used   (Mb)   
Ncells   10669997   569.9   27719894  1480.5   27719894  1480.5
Vcells 4711983266 35949.6 8794373681 67095.8 8048195828 61402.9

#### Mean Inf Tree 0

In [29]:
treeMeanInf0 <- mergeTree(treesMeanInf0, se = mapDf)
mb <- mergeLeaves(treeMeanInf0, seChimp[txpsAll,])
treeMeanInf0 <- mb[["tree"]]
yMeanInfNS0 <- mb[["ySwish"]]
rm(mb)
gc()
sum(rownames(yMeanInfNS0) != treeMeanInf0$tip)

yAggMeanInfNS0 <- prepSwish(treeMeanInf0, seChimp[treeMeanInf0$tip,])
yAggMeanInfNS0 <- fishpond::computeInfRV(yAggMeanInfNS0, meanVariance=F)

yAggMeanInf0 <- prepSwish(treeMeanInf0, seChimpScaled[treeMeanInf0$tip,])

infReps <- assays(yAggMeanInf0)[grep("infRep", assayNames(yAggMeanInf0))]
infReps <- abind::abind(as.list(infReps), along = 3)
mcols(yAggMeanInfNS0)[["LFC"]] <- getLog2FC(infReps, colData(yAggMeanInf0)[["condition"]])

rm(yAggMeanInf0)
gc()

save(yAggMeanInfNS0, file="environment/Chimp/mode=gcbias/post_type=gibbs_nrep=100_tf=100/tree_analysis/yAggMeanInfNS0.RData")
save(treeMeanInf0, file="environment/Chimp/mode=gcbias/post_type=gibbs_nrep=100_tf=100/tree_analysis/treeMeanInf0.RData")

[1] "Missing txps 5519"


used       (Mb)    gc trigger (Mb)    max used   (Mb)   
Ncells   10667523   569.8   27719894  1480.5   27719894  1480.5
Vcells 3278129542 25010.2 6107081724 46593.4 5595482190 42690.2

[1] 0

used       (Mb)    gc trigger (Mb)    max used   (Mb)   
Ncells   10668378   569.8   27719894  1480.5   27719894  1480.5
Vcells 3798288646 28978.7 7328578068 55912.7 7325005957 55885.4

#### Correlation

In [28]:
mb <- mergeLeaves(trMeanCorUPGMA, seChimp[txpsAll,])
treeCor <- mb[["tree"]]
rm(mb)
gc()

yAggCorNS <- prepSwish(treeCor, seChimp[treeCor$tip,])
yAggCorNS <- fishpond::computeInfRV(yAggCorNS, meanVariance=F)

yAggCor <- prepSwish(treeCor, seChimpScaled[treeCor$tip,])

infReps <- assays(yAggCor)[grep("infRep", assayNames(yAggCor))]
infReps <- abind::abind(as.list(infReps), along = 3)

mcols(yAggCorNS)[["LFC"]] <- getLog2FC(infReps, colData(yAggCor)[["condition"]])
rm(yAggCor)
gc()

save(yAggCorNS, file="environment/Chimp/mode=gcbias/post_type=gibbs_nrep=100_tf=100/tree_analysis/yAggCorNS.RData")
save(treeCor, file="environment/Chimp/mode=gcbias/post_type=gibbs_nrep=100_tf=100/tree_analysis/treeCor.RData")

used       (Mb)    gc trigger (Mb)    max used   (Mb)   
Ncells   10666619   569.7   27719894  1480.5   27719894  1480.5
Vcells 2931805877 22368.0 6107081724 46593.4 5595482190 42690.2

#### Threshold

In [26]:
treeThr <- mergeTree(treesThr, se = mapDf)
mb <- mergeLeaves(treeThr, seChimp[txpsAll,])
treeThr<- mb[["tree"]]
yThrNS <- mb[["ySwish"]]
rm(mb)
gc()
sum(rownames(yThrNS) != treeThr$tip)

yAggThrNS <- prepSwish(treeThr, seChimp[treeThr$tip,])
yAggThrNS <- fishpond::computeInfRV(yAggThrNS, meanVariance=F)

yAggThr <- prepSwish(treeThr, seChimpScaled[treeThr$tip,])

infReps <- assays(yAggThr)[grep("infRep", assayNames(yAggThr))]
infReps <- abind::abind(as.list(infReps), along = 3)
mcols(yAggThrNS)[["LFC"]] <- getLog2FC(infReps, colData(yAggThrNS)[["condition"]])

rm(yAggThr)
gc()

save(yAggThrNS, file="environment/Chimp/mode=gcbias/post_type=gibbs_nrep=100_tf=100/tree_analysis/yAggThrNS.RData")
save(treeThr, file="environment/Chimp/mode=gcbias/post_type=gibbs_nrep=100_tf=100/tree_analysis/treeThr.RData")

used       (Mb)    gc trigger (Mb)    max used   (Mb)   
Ncells   10640768   568.3   27719894  1480.5   27719894  1480.5
Vcells 2120494503 16178.1 4240906754 32355.6 4239696840 32346.4

In [ ]:
reshape::cast(dfInfRV %>% filter(depth!=1) %>% group_by(type,modDep), type~modDep, max, value="infRV")

In [14]:
save_dir <- "environment/Chimp/mode=gcbias/post_type=gibbs_nrep=100_tf=100/tree_analysis"
### Threshold
#### NGenes
nNodes <- length(treeThr$tip)+1:treeThr$Nnode
desc <- Descendants(treeThr, nNodes)
genesThr <- mclapply(desc, function(txps) unique(mapDf[treeThr$tip.label[txps],"gene_id"]), mc.cores=1)
save(genesThr, file=file.path(save_dir,"genesThr.RData"))

In [ ]:
#### NGenes
nNodes <- length(treeCons$tip)+1:treeCons$Nnode
desc <- Descendants(treeCons, nNodes)
genesCons <- mclapply(desc, function(txps) unique(mapDf[treeCons$tip.label[txps],"gene_id"]), mc.cores=1)
save(genesCons, file=file.path(dir,"genesCons.RData"))

In [ ]:
#### NGenes
nNodes <- length(treeCons0$tip)+1:treeCons0$Nnode
desc <- Descendants(treeCons0, nNodes)
genesCons0 <- mclapply(desc, function(txps) unique(mapDf[treeCons0$tip.label[txps],"gene_id"]), mc.cores=1)
save(genesCons0, file=file.path(save_dir,"genesCons0.RData"))

In [15]:
#### NGenes
nNodes <- length(treeMeanInf0$tip)+1:treeMeanInf0$Nnode
desc <- Descendants(treeMeanInf0, nNodes)
genesMeanInf0 <- mclapply(desc, function(txps) unique(mapDf[treeMeanInf0$tip.label[txps],"gene_id"]), mc.cores=1)
save(genesMeanInf0, file=file.path(save_dir,"genesMeanInf0.RData"))

In [ ]:
#### NGenes
nNodes <- length(treeCor$tip)+1:treeCor$Nnode
desc <- Descendants(treeCor, nNodes)
genesCor <- mclapply(desc, function(txps) unique(mapDf[treeCor$tip.label[txps],"gene_id"]), mc.cores=1)
save(genesCor, file=file.path(save_dir,"genesCor.RData"))